In [915]:
from datetime import datetime
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [916]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir('bases/'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'29.08.2024'

In [917]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.09.01


'reporte\\2024.09.01'

In [918]:
base_pagos_path = 'bases/Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'bases/base_asignacion.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)

bases/Base Pagos 29.08.2024.xlsx
bases/base_asignacion.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.01\MONOPRODUCTO 29.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.01\MULTIPRODUCTO 29.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.01\REACTIVA 29.08.2024.xlsx
c:\Users\miria\Documents\BBVA\GitHub\base-pagos\reporte\2024.09.01\NO ENVIADOS 29.08.2024.xlsx


In [919]:
#################### BASE DE PAGOS ####################

In [920]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(459, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-28,29251432,248,321,3496.67,PEN,EMPRESA GUIAME SE･OR DE MURUHUAY EIRL,P
1,11,2024-08-28,23419268,248,50,5460.00,PEN,IVAN ROBINSON VEGA PARRA,P
2,11,2024-08-28,24323662,248,235,549.00,PEN,MARCOS MANUEL SANCHEZ PANTOJA,P
3,11,2024-08-28,28447023,248,235,550.00,PEN,ROLFI FIDEL MANTARI PASTRANA,P
4,11,2024-08-28,26736347,248,113,2000.00,PEN,A & W INDUSTRIAL SAC,P


In [921]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(459, 10)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-08-28,29251432,00248,321,3496.67,PEN,EMPRESA GUIAME SE･OR DE MURUHUAY EIRL,P,01-Sep,MIV
1,2024-08-28,23419268,00248,50,5460.00,PEN,IVAN ROBINSON VEGA PARRA,P,01-Sep,MIV
2,2024-08-28,24323662,00248,235,549.00,PEN,MARCOS MANUEL SANCHEZ PANTOJA,P,01-Sep,MIV
3,2024-08-28,28447023,00248,235,550.00,PEN,ROLFI FIDEL MANTARI PASTRANA,P,01-Sep,MIV
4,2024-08-28,26736347,00248,113,2000.00,PEN,A & W INDUSTRIAL SAC,P,01-Sep,MIV


In [922]:
df_base.isnull().sum()

FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [923]:
#################### ASIGNACION ####################

In [924]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)


In [925]:
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 9)
codigo               int64
ID_FLAG              int64
cont_18              int64
nombre              object
TIPO_CARTERA        object
TIPO_FONDO          object
clave               object
AGENCIA CORRECTA    object
ID_VTA.             object
dtype: object


,codigo,ID_FLAG,cont_18,nombre,TIPO_CARTERA,TIPO_FONDO,clave,AGENCIA CORRECTA,ID_VTA.
0,30739481,2,1100109600055161,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
1,30739481,2,1100109600057679,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
2,21874900,4,1100505000360222,JULIA ANGELICA SOTO COTRINA,UNSECURED,NaN,BC,ASESCOM RJ,-
3,30293392,1,1101305002987960,INNOVA FASHION GROUPSAC,UNSECURED,NaN,BC,CLASA MORA,-
4,20687681,1,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,UNSECURED,NaN,BC,RodriguezAngobaldo,-


In [926]:
#################### BACKUPS ####################

In [927]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (459, 10)
Asignacion (242511, 9)


In [928]:
base_count = df_base_test.shape[0]
base_count

459

In [929]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})

df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(459, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-08-28,29251432,00248,321,3496.67,PEN,EMPRESA GUIAME SE･OR DE MURUHUAY EIRL,P,2,001103219600546204,REACTIVA,BC,EMPRESA GUIAME SEÐORDE MURUHUAY EIRL,01-Sep,MIV,UNSECURED,MORNESE MORA
2,2024-08-28,23419268,00248,50,5460.00,PEN,IVAN ROBINSON VEGA PARRA,P,<NA>,NaN,NULL,NaN,NaN,01-Sep,MIV,NULL,NaN
3,2024-08-28,24323662,00248,235,549.00,PEN,MARCOS MANUEL SANCHEZ PANTOJA,P,1,001102355002710498,NULL,KSTBC,MARCOS MANUEL SANCHEZ PANTOJA,01-Sep,MIV,UNSECURED,MORNESE MORA
4,2024-08-28,28447023,00248,235,550.00,PEN,ROLFI FIDEL MANTARI PASTRANA,P,1,001108149601531413,NULL,KSTBC,ROLFI FIDEL MANTARI PASTRANA,01-Sep,MIV,UNSECURED,CLASA MORA
5,2024-08-28,26736347,00248,113,2000.00,PEN,A & W INDUSTRIAL SAC,P,2,001101039600277054,REACTIVA,BC,A & W INDUSTRIAL SAC,01-Sep,MIV,UNSECURED,MORNESE MORA


In [930]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [931]:
df_base_test.shape

(459, 17)

In [932]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(29, 17)

In [933]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

29

In [934]:
flag_count = base_count - no_flag_count
flag_count

430

In [935]:
#################### MONOPRODUCTO ####################

In [936]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(348, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
3,2024-08-28,24323662,00248,235,549.0,PEN,MARCOS MANUEL SANCHEZ PANTOJA,P,1,001102355002710498,NULL,KSTBC,MARCOS MANUEL SANCHEZ PANTOJA,01-Sep,MIV,UNSECURED,MORNESE MORA
4,2024-08-28,28447023,00248,235,550.0,PEN,ROLFI FIDEL MANTARI PASTRANA,P,1,001108149601531413,NULL,KSTBC,ROLFI FIDEL MANTARI PASTRANA,01-Sep,MIV,UNSECURED,CLASA MORA
7,2024-08-28,30016377,00248,253,3500.0,PEN,AARON ANIBAL RAMIREZ HERRERA,P,1,001104085012680376,NULL,EXJ,AARON ANIBAL RAMIREZ HERRERA,01-Sep,MIV,UNSECURED,CLASA MORA
8,2024-08-28,24214222,00248,179,2500.0,PEN,BETTY ANTONIA SANCHEZ BORDA...,P,1,001107169600096613,NULL,BC,BETTY ANTONIA SANCHEZ BORDA,01-Sep,MIV,UNSECURED,CLASA MORA
9,2024-08-28,31294224,00248,267,750.0,PEN,CARLOS SIRLUPU MAZA 28/08/2024,P,1,001105045004419466,NULL,EXJ,CARLOS DANIEL SIRLUPU MAZA,01-Sep,MIV,UNSECURED,MORNESE MORA


In [937]:
mono_count = df_mono.shape[0]
mono_count

348

In [938]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    326
SECURED       22
Name: count, dtype: int64

In [939]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        337
REACTIVA     11
Name: count, dtype: int64

In [940]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          319
SECURED       NULL           18
UNSECURED     REACTIVA        7
SECURED       REACTIVA        4
Name: count, dtype: int64

In [941]:
#################### MONO REACTIVA ####################

In [942]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] != 'NULL')]
df_reactiva.sort_values(by=['CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(7, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-08-28,22357991,00248,260,3000.0,PEN,ANIOR IMPORT MEDIC EIRL...,P,1,001102609600335431,REACTIVA,BC,ANIOR IMPORT MEDIC EIRL,01-Sep,MIV,UNSECURED,ASESCOM RJ
1,2024-08-28,24748534,00248,643,4000.0,PEN,JUAN ABEL ZAVALA AGUILAR,P,1,001106439600068567,REACTIVA,BC,INVERSIONES Y MULTISERVICIOS GRISEL SAC,01-Sep,MIV,UNSECURED,CLASA MORA
2,2024-08-28,25625797,00248,177,500.0,PEN,MARTIN TOLENTINO ILUMINACION PROFESIONA.,P,1,001101759600228463,REACTIVA,BC,MARTIN TOLENTINO ILUMINACION PROFESIONALEIRL,01-Sep,MIV,UNSECURED,CLASA MORA
3,2024-08-28,26762974,00248,228,1500.0,PEN,"DISTRIBUCIONES Y SERVICIOS DDYMM CONSTR,",P,1,001102289600352366,REACTIVA,BC,DISTRIBUCIONES & SERVICIOS DDYMM CONSTRUYE SRL,01-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-08-28,26959068,00248,628,1500.0,PEN,OSCAR LLANCO INGARUCA,P,1,001106289600038682,REACTIVA,BC,OSCAR ANSELMO LLANCO INGARUCA,01-Sep,MIV,UNSECURED,ASESCOM RJ


In [943]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

7

In [944]:
df_reactiva.to_excel(reactiva, index=False)

In [945]:
#################### FIN REACTIVA ####################

In [946]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

22

In [947]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)

print(df_mono.shape)
df_mono.head(5)

(319, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-08-28,06989136,00248,735.0,PEN,JORGE ROBERTO LOPEZ VINATEA..,P,1,001105045002739250,JORGE ROBERTO LOPEZ VINATEA,01-Sep,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
1,2024-08-28,09838775,00248,110.0,PEN,08470561*,P,1,001101745001861173,ESPERANZA ORDINOLA ZEGARRA,01-Sep,MIV,UNSECURED,NULL,EXJ,CLASA MORA
2,2024-08-28,11193064,00248,1000.0,PEN,CARMEN DEL PILAR CORNEJO,P,1,001105045001029721,CARMEN DEL PILAR CORNEJO TELLO,01-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
3,2024-08-28,11929451,00248,96.0,PEN,YENNY ROXANA STUVA LOPEZ,P,1,001103585001566163,YENNY ROXANA STUVA LOPEZ,01-Sep,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
4,2024-08-28,13844240,00248,2100.0,PEN,JAIME KARLO BEDOYA MENDOZA - 29628878.,P,1,001105045003931503,JAIME KARLO BEDOYA MENDOZA,01-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ


In [948]:
df_mono.to_excel(monoproducto, index=False)

In [949]:
mono_count_final = df_mono.shape[0]
mono_count_final

319

In [950]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [951]:
#################### FIN MONOPRODUCTO ####################

In [952]:
#################### MULTIPRODUCTO ####################

In [953]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(82, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-08-28,29251432,00248,321,3496.67,PEN,EMPRESA GUIAME SE･OR DE MURUHUAY EIRL,P,2,001103219600546204,REACTIVA,BC,EMPRESA GUIAME SEÐORDE MURUHUAY EIRL,01-Sep,MIV,UNSECURED,MORNESE MORA
5,2024-08-28,26736347,00248,113,2000.00,PEN,A & W INDUSTRIAL SAC,P,2,001101039600277054,REACTIVA,BC,A & W INDUSTRIAL SAC,01-Sep,MIV,UNSECURED,MORNESE MORA
11,2024-08-28,26350069,00248,281,300.00,PEN,CARLOS AURELIO ADRIANZEN DELGADO..,P,2,001108149603189763,NULL,BC,CARLOS AURELIO ADRIANZEN DELGADO,01-Sep,MIV,UNSECURED,ASESCOM RJ
19,2024-08-28,21785140,00248,237,1830.00,PEN,AUGUSTO VEGA PANTOJA,P,3,001108149602931253,NULL,EXJ,AUGUSTO JOSE VEGA PANTOJA,01-Sep,MIV,UNSECURED,ASESCOM RJ
22,2024-08-28,30518871,00248,621,3688.00,PEN,AGROINDUSTRIAS MOKEWUA SOCIEDA,P,4,001104379601425983,NULL,KSTBC,AGROINDUSTRIAS MOKEWUA SOCIEDAD COMERCIAL DE R...,01-Sep,MIV,UNSECURED,ASESCOM RJ


In [954]:
multi_count = df_multi.shape[0]
multi_count

82

In [955]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    78
SECURED       4
Name: count, dtype: int64

In [956]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        72
REACTIVA    10
Name: count, dtype: int64

In [957]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

4

In [958]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

df_multi.sort_values(by=['CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)

print(df_multi.shape)
df_multi.head(5)

(78, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-08-28,00669822,00248,500.0,PEN,PRESTAMO28,P,2,None,NULL,None,CARLOS ESCUDERO RIOS,01-Sep,MIV
1,2024-08-28,11570828,00248,2500.0,PEN,GREGORIO GILMER ROSALES ROJAS,P,2,None,NULL,None,GREGORIO GILMER ROSALES ROJAS,01-Sep,MIV
2,2024-08-28,20062652,00248,1385.0,PEN,PEDRO ANTONIO HUAPAYA CABRERA,P,2,None,NULL,None,PEDRO ANTONIO HUAPAYA CABRERA,01-Sep,MIV
3,2024-08-28,20665952,00248,3500.0,PEN,JULIO CESAR ARCE CHIRINOS,P,2,None,NULL,None,JULIO CESARARCECHIRINOS,01-Sep,MIV
4,2024-08-28,20753079,00248,1000.0,PEN,JUAN JOSE ROMERO PACHECO-22302657,P,3,None,NULL,None,JUAN JOSE ROMERO PACHECO,01-Sep,MIV


In [959]:
df_multi.to_excel(multiproducto, index=False)

In [960]:
multi_count_final = df_multi.shape[0]
multi_count_final

78

In [961]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [962]:
#################### FIN MULTIPRODUCTO ####################

In [963]:
#################### NO ENVIADOS ####################

In [964]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 55
Enviados: 404
Total: 459


In [965]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [966]:
print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')


In [967]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['TIPO_CARTERA'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_CARTERA'], axis=1)

df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype(str).str.replace('0', '', regex=True)

df_no_enviados.sort_values(by=['CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(55, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-08-28,09005560,00248,106,3000.0,PEN,CELIA NAVARRO MENDIZABAL,P,1,001107279600317792,NULL,BC,CELIA NAVARRO MENDIZABAL,01-Sep,MIV,SECURED,SIN AGENCIA
1,2024-08-28,20176809,00248,847,9423.0,PEN,GIOVANNI ALEJANDRO MARTINEZ LA TORRE,P,,None,None,NaN,NaN,01-Sep,MIV,None,NaN
2,2024-08-28,20593175,00248,348,300000.0,PEN,DESTILERIA NAYLAMP EIRL-,P,1,001104029600110163,NULL,BC,DESTILERIA NAYLAMP EIRL,01-Sep,MIV,SECURED,SIN AGENCIA
3,2024-08-28,20689141,00248,242,6000.0,PEN,LUZZ VILCAA LOPEZZ,P,1,001102429600050129,NULL,BC,LUZ AMPARO VILCA LOPEZ,01-Sep,MIV,SECURED,SIN AGENCIA
4,2024-08-28,20708611,00248,314,6686.0,PEN,PAGO TC 6879,P,,None,None,NaN,NaN,01-Sep,MIV,None,NaN


In [968]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [969]:
df_no_enviados.to_excel(no_enviados_path, index=False)

In [970]:
#################### FORMATO EXCEL ####################

In [971]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'instruccion':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 13): # Columnas K-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 17):  # Columnas M-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(17, 18):  # Columna Q
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [972]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)
os.startfile(no_enviados_path)

In [973]:
#################### AGENCIAS ####################